<a href="https://colab.research.google.com/github/rajpabari/gflownets-rlhf/blob/main/Generate_label_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/rajpabari/gflownets-rlhf/
import pandas as pd
import numpy as np
from itertools import combinations

fatal: destination path 'gflownets-rlhf' already exists and is not an empty directory.


In [36]:
train_data=pd.read_csv("gflownets-rlhf/data/train.csv")
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [37]:
def clc(dataframe, weights, columns, new_column_name):
    # Calculate the linear combination
    linear_combination = np.dot(dataframe[columns], weights)
    
    # Add the new column to the DataFrame
    dataframe[new_column_name] = linear_combination
    
    return dataframe
train_data = clc(train_data,[0.5,1,0.5,1,0.5,1],train_data.columns[2:],"score")
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0.0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0.0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0.0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0.0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0.0


In [38]:
train_data.index

RangeIndex(start=0, stop=159571, step=1)

In [42]:
from tqdm import tqdm
import itertools
import random
def create_pairwise_dataset(df,max_size=80000):
    # Create all pairwise combinations of comments
    pool = np.random.permutation(df.index.stop)
    pool2 = np.random.permutation(df.index.stop)
    # Initialize lists to store the data
    pairs = []
    labels = []
    print(pool.shape,df.index.stop)
    # Iterate through the combinations
    k=0
    with tqdm(total=max_size) as pbar:
      while len(pairs) < max_size:
          k+=1
          if k == df.index.stop-1:
              pool = np.random.permutation(df.index.stop)
              pool2 = np.random.permutation(df.index.stop)
              k=0
          #print(k,pool[k],pool2[k])
          i=pool[k]
          j=pool2[k]
          comment_i = df.loc[i, 'comment_text']
          comment_j = df.loc[j, 'comment_text']
          score_i = df.loc[i, 'score']
          score_j = df.loc[j, 'score']
          
          # Determine the label based on the scores
          if score_i < score_j:
              label = 1
          elif score_i > score_j:
              label = 0
          else:
            continue
          pairs.append((comment_i, comment_j, label))
          pbar.update(1)
    
    return  pd.DataFrame(pairs, columns=["comment1","comment2","score"])
train_df = create_pairwise_dataset(train_data)
train_df.head()

(159571,) 159571


100%|██████████| 80000/80000 [00:16<00:00, 4725.23it/s]


,comment1,comment2,score
0,I dont give a damn anymore.\nheres a personal ...,Further note to potential reviewing admin If w...,0
1,"""\n\n File:2007 jersey.jpg listed for deletion...",Wikipedia:Headless Chickens \n\nI have nominat...,1
2,"""\nChristians\nWhat about all the idiot Christ...","""\n\n Different from \n\nis a common gramatica...",0
3,SUCK MY PLUMS YOU STUPID SEPTIC CUNT,""" January 2009 (UTC)\nEli is physically a boy ...",0
4,2K \n\nA Kenyan rap and hip-hop artist.,""" 2006 (UTC)\n\n On notability \n\nOne admin d...",1


In [43]:
train_df.to_csv("./train_reward.csv")

In [44]:
import gc
gc.collect()

0

In [47]:
test_data=pd.read_csv("gflownets-rlhf/data/test.csv")
test_labels=pd.read_csv("gflownets-rlhf/data/test_labels.csv")
test_data = test_data[(test_labels['toxic'] >= 0) & (test_labels['severe_toxic'] >= 0) & (test_labels['obscene'] >= 0) & (test_labels['threat'] >= 0) & (test_labels['insult'] >= 0) & (test_labels['identity_hate'] >= 0)]
test_labels = test_labels[(test_labels['toxic'] >= 0) & (test_labels['severe_toxic'] >= 0) & (test_labels['obscene'] >= 0) & (test_labels['threat'] >= 0) & (test_labels['insult'] >= 0) & (test_labels['identity_hate'] >= 0)]
test_data.head(), test_labels.head()

(                  id                                       comment_text
 5   0001ea8717f6de06  Thank you for understanding. I think very high...
 7   000247e83dcc1211                   :Dear god this site is horrible.
 11  0002f87b16116a7f  "::: Somebody will invariably try to add Relig...
 13  0003e1cccfd5a40a  " \n\n It says it right there that it IS a typ...
 14  00059ace3e3e9a53  " \n\n == Before adding a new product to the l...,
                   id  toxic  severe_toxic  obscene  threat  insult  \
 5   0001ea8717f6de06      0             0        0       0       0   
 7   000247e83dcc1211      0             0        0       0       0   
 11  0002f87b16116a7f      0             0        0       0       0   
 13  0003e1cccfd5a40a      0             0        0       0       0   
 14  00059ace3e3e9a53      0             0        0       0       0   
 
     identity_hate  
 5               0  
 7               0  
 11              0  
 13              0  
 14              0  )

In [48]:
test_df = pd.concat([test_data.set_index('id'), test_labels.set_index('id')], axis=1).reset_index()
test_df = clc(test_df,[0.5,1,0.5,1,0.5,1],test_df.columns[2:],"score")
test_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,0.0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,0.0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,0.0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,0.0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,0.0


In [50]:
test_df = create_pairwise_dataset(test_df, max_size=10000)
test_df.head()

(63978,) 63978


100%|██████████| 10000/10000 [00:02<00:00, 4977.15it/s]


,comment1,comment2,score
0,Inactive how? They never broke up. In 2000 and...,""" \n\n == WikiProject Queensland/To-do == \n\n...",0
1,i hate black poeple...mia coleman is a bitchhh...,::Thanks for the xplanations. Cheers. -,0
2,"fuck you fucking pussy, go get laid, oh im sor...",""" \n\n *Ridiculous! They're selling sights as ...",0
3,""", 13 August 2008 (UTC) \n :In all those insta...",== Somalia nigger == [soh-mah-lee-uh nig-er] \...,1
4,This article is rubbish,Please more information about this city.,0


In [51]:
test_df.to_csv("./test_reward.csv")